# Quantum States and Gates Exploration

This notebook tests the functionality of quantum statevector simulators and quantum gate simulators.

In [518]:
import numpy as np
import numpy.typing as npt
from random import random

## Qubits

Qubits should be able to:
1. Store complex amplitudes
2. Support measurement
3. Evolve according to quantum operations

### Utilities

The absolute value of a number squared will likely be used often to find the probabilities of measurement outcomes.

The Euclidean norm of a vector will likely be used often to check the validity of a statevector or a unitary matrix.

The length of a statevector must be equal to a power of 2

In [519]:
def abs_squared(num: complex | float):
    return np.abs(np.square(num))

def e_norm(vector: npt.NDArray):
    return np.sum(abs_squared(vector))

def is_pow_of_2(length: int):
    while length % 2 == 0:
        length //= 2
    return length == 1

### Version 1

The `Qubit` class will keep track of individual qubits, including their initialization, measurements, probabilities, and evolutions.

In [520]:
class Qubit:
    def __init__(self, alpha: complex, beta: complex):
        if e_norm([alpha, beta]) != 1:
            print(e_norm([alpha, beta]))
            raise Exception("The Euclidean norm of the state must be 1")
        
        self.alpha = alpha
        self.beta = beta

    def __str__(self):
        return f"{self.alpha}|0⟩ + {self.beta}|1⟩"
    
    def measure(self):
        if random() < abs_squared(self.alpha):
            self.alpha = 1
            self.beta = 0
            return 0
        else:
            self.alpha = 0
            self.beta = 1
            return 1
        
    def evolve(self, operator: npt.NDArray):
        if operator.shape != (2, 2):
            raise Exception(f"The operator's shape {operator.shape} is incompatible with the qubit")
        state = np.ndarray([self.alpha, self.beta])
        transformed = operator @ state
        self.alpha = transformed[0]
        self.beta = transformed[1]

    def to_numpy(self):
        return np.ndarray([self.alpha, self.beta])
        

### Testing `Qubit`

Testing reveals that `e_norm` may result in rounding errors 

In [521]:
# This test fails because the Euclidean norm is not exactly 1
# q = Qubit(1/np.sqrt(2), 1/np.sqrt(2))

### Version 2

Add tolerance to Euclidean norm checks and rounding to string representation

In [522]:
class Qubit:
    def __init__(self, alpha: complex, beta: complex):
        if abs(1 - e_norm([alpha, beta])) > 1e-5:
            raise Exception("The Euclidean norm of the state must be 1")
        
        self.alpha = alpha
        self.beta = beta

    def __str__(self):
        return f"{self.alpha:.4f}|0⟩ + {self.beta:.4f}|1⟩"
    
    def measure(self):
        if random() < abs_squared(self.alpha):
            self.alpha = 1
            self.beta = 0
            return 0
        else:
            self.alpha = 0
            self.beta = 1
            return 1
        
    def evolve(self, operator: npt.NDArray):
        if operator.shape != (2, 2):
            raise Exception(f"The operator's shape {operator.shape} is incompatible with the qubit")
        state = np.array([self.alpha, self.beta])
        transformed = operator @ state
        self.alpha = transformed[0]
        self.beta = transformed[1]

    def to_numpy(self):
        return np.array([self.alpha, self.beta])
        

### Testing `Qubit`

These tests show that the initialization, measurement, and string representation functions are working properly

In [523]:
q = Qubit(1/np.sqrt(2), 1/np.sqrt(2))
print(q)

q = Qubit(1, 0)
print(q)

q = Qubit(0.5, complex(0, np.sqrt(3) / 2))
print(q)

print(q.measure())
print(q)

count = {
    0: 0,
    1: 0
}
for i in range(1000):
    q = Qubit(0.5, complex(0, np.sqrt(3) / 2))
    measurement = q.measure()
    count[measurement] += 1

# 0 should be measured 25% of the time
# 1 should be measured 75% of the time
print(count)

0.7071|0⟩ + 0.7071|1⟩
1.0000|0⟩ + 0.0000|1⟩
0.5000|0⟩ + 0.0000+0.8660j|1⟩
1
0.0000|0⟩ + 1.0000|1⟩
{0: 239, 1: 761}


## Operators

Operators should be able to:
1. Transform qubits and larger statevectors

### Version 1

The `Operator` class will keep track of information about the operator and ensure that it is a unitary matrix.

In [524]:
class Operator:
    def __init__(self, matrix: npt.NDArray | list):
        matrix = np.array(matrix, dtype=complex)
        if matrix.shape[0] != matrix.shape[1]:
            raise Exception("The operation must be a square matrix")
        if not np.allclose((matrix @ np.conjugate(matrix).T), np.identity(matrix.shape[0]), atol=1e-5):
            raise Exception("The operation must be a unitary matrix")
        self.operation = matrix
    
    def to_numpy(self):
        return self.operation
    
    def __str__(self):
        return str(self.operation.round(4))

### Testing `Operator`

These tests show that the initialization is working.

In [525]:
h = Operator(np.array([
    [1, 1],
    [1, -1]
]) / np.sqrt(2))

print(h)


[[ 0.7071+0.j  0.7071+0.j]
 [ 0.7071+0.j -0.7071+0.j]]


### Class Revision: Qubit (v3)

**Reason for update:**

The original implementation did not include account for the `Operator` class in the `evolve` method. The data type for the parameter must be `Operator` to ensure that the argument is a unitary matrix.

**Changes made:**
- Changed parameter type to `Operator` in `evolve` method
- Added conversion from operator to NumPy

In [526]:
class Qubit:
    def __init__(self, alpha: complex, beta: complex):
        if abs(1 - e_norm([alpha, beta])) > 1e-5:
            raise Exception("The Euclidean norm of the state must be 1")
        
        self.alpha = alpha
        self.beta = beta

    def __str__(self):
        return f"{self.alpha:.4f}|0⟩ + {self.beta:.4f}|1⟩"
    
    def measure(self):
        if random() < abs_squared(self.alpha):
            self.alpha = 1
            self.beta = 0
            return 0
        else:
            self.alpha = 0
            self.beta = 1
            return 1
        
    def evolve(self, operator: Operator):
        operator = operator.to_numpy()
        if operator.shape != (2, 2):
            raise Exception(f"The operator's shape {operator.shape} is incompatible with the qubit")
        state = np.array([self.alpha, self.beta])
        transformed = operator @ state
        self.alpha = transformed[0]
        self.beta = transformed[1]

    def to_numpy(self):
        return np.array([self.alpha, self.beta])
        

### Testing `Qubit.evolve()`

These tests demonstrate that qubits are correctly transformed by operators

In [527]:
q = Qubit(1, 0)
print(q)
X = Operator([
    [0, 1],
    [1, 0]
])
q.evolve(X)
print(q)

q = Qubit(0, 1)
print(q)
H = Operator([
    [1, 1],
    [1, -1]
] / np.sqrt(2))
q.evolve(H)
print(q)

1.0000|0⟩ + 0.0000|1⟩
0.0000+0.0000j|0⟩ + 1.0000+0.0000j|1⟩
0.0000|0⟩ + 1.0000|1⟩
0.7071+0.0000j|0⟩ + -0.7071+0.0000j|1⟩


## Quantum Circuits

Quantum circuits should be able to:
1. Keep track of multiple qubits and create a single statevector representation
2. Perform a series of operations on the qubits
3. Update each individual qubit according to any operations or measurements

### Version 1

The `QuantumState` class will store a number of qubits in statevector and update each qubit according to any operations or measurements.

In [528]:
class QuantumCircuit:
    def __init__(self):
        self.register: dict[str: Qubit] = {}
        self.statevector: npt.NDArray = np.array([])

    def add_qubit(self, qubit: Qubit, name: str):
        self.register[name] = qubit
        if len(self.statevector) == 0:
            self.statevector = qubit.to_numpy()
        else:
            self.statevector = np.kron(
                self.statevector, qubit.to_numpy()
            )
            

    def measure(self, qubit: str):
        if qubit in self.register.keys:
            measurement = self.register[qubit].measure()
            return measurement
        else:
            raise Exception(f"There is no qubit named {qubit} in the circuit")

    def __str__(self):
        return self.statevector.__str__()

In [529]:
a = Qubit(1, 0)
q = Qubit(1/np.sqrt(2), 1/np.sqrt(2))
circuit = QuantumCircuit()
circuit.add_qubit(a, "A")
circuit.add_qubit(q, "Q")

print(circuit)

[0.70710678 0.70710678 0.         0.        ]


## !! Change of Plans !!

I realized that due to entanglement, it is impossible to describe qubits independently once it is entangled with others. Because of this, I will transform the `Qubit` class to a `QuantumState` class that will keep track of any number of qubits in a statevector.

## Quantum States

A quantum state should be able to:
1. Keep track of multiple qubits in a single statevector
2. Perform a series of operations on the qubits
3. Update the statevector depending on any partial measurements

### Version 1

The `QuantumState` class will take in a number of qubits, represent them through a statevector, and implement functionality for operators and partial measurements. For simplicity, some common operators will be built into the class.

In [534]:
class QuantumState:
    def __init__(self, statevector: npt.NDArray):
        if not is_pow_of_2(len(statevector)):
            raise Exception("The length of the statevector must be a power of 2")
        if abs(1 - e_norm(statevector)) > 1e-5:
            raise Exception("The Euclidean norm of the state must be 1")
        
        self.state = np.array(statevector, dtype=complex)
        self.n_qubits = round(np.log2(len(statevector)))

    def __str__(self):
        return self.state.round(4).__str__()
    
    @classmethod
    def from_qubits(self, *qubits: 'npt.NDArray | QuantumState'):
        statevector = np.array([1], dtype=complex)
        for qubit in qubits:
            if isinstance(qubit, QuantumState):
                statevector = np.kron(statevector, qubit.to_numpy())
            else:
                statevector = np.kron(statevector, qubit)

        return self(statevector)
    
    def add_qubit(self, qubit: 'npt.NDArray | QuantumState'):
        if isinstance(qubit, QuantumState):
            q = qubit.to_numpy()
            self.state = np.kron(self.state, q)
            self.n_qubits += round(np.log2(len(q)))
        else:
            self.state = np.kron(self.state, qubit)
            self.n_qubits += round(np.log2(len(qubit)))
    
    def measure(self, index: int):
        if index < 0 or index >= self.n_qubits:
            raise Exception(f"Qubit index {index} is out of range for {self.n_qubits} qubits")

        index = self.n_qubits - index - 1
        probs = np.abs(np.square(self.state))

        prob_0 = sum(probs[i] for i in self.get_qubit_indices(index, 0))
        result = 0 if random() < prob_0 else 1

        indices = self.get_qubit_indices(index, result)
        mask = np.zeros_like(self.state)
        mask[indices] = 1
        self.state *= mask
        self.state /= np.sqrt(sum(probs[indices]))
        return result

    def get_qubit_indices(self, index: int, outcome: int):
        indices = []
        for i in range(len(self.state)):
            bits = format(i, f'0{self.n_qubits}b')
            if int(bits[index]) == outcome:
                indices.append(i)
        return indices
    
    def CNOT(self, control: int, target: int):
        control = self.n_qubits - control - 1
        target = self.n_qubits - target - 1
        new_state = np.zeros_like(self.state)
        for i in range(len(self.state)):
            b = format(i, f'0{self.n_qubits}b')
            if b[control] == '1':
                flipped = list(b)
                flipped[target] = '0' if b[target] == '1' else '1'
                j = int(''.join(flipped), 2)
                new_state[j] += self.state[i]
            else:
                new_state[i] = self.state[i]
        self.state = new_state
        
    def X(self, index: int):
        index = self.n_qubits - index - 1
        x = np.array([
            [0, 1],
            [1, 0]
        ])
        operator = self.__expand_operator(index, x)
        self.state = operator @ self.state

    def Z(self, index: int):
        index = self.n_qubits - index - 1
        z = np.array([
            [1, 0],
            [0, -1]
        ])
        operator = self.__expand_operator(index, z)
        self.state = operator @ self.state

    def H(self, index: int):
        index = self.n_qubits - index - 1
        h = np.array([
            [1, 1],
            [1, -1]
        ]) / np.sqrt(2)
        operator = self.__expand_operator(index, h)
        self.state = operator @ self.state

    def __expand_operator(self, index: int, operation: npt.NDArray):
        i_2 = np.identity(2)
        if index == 0:
            operator = operation
        else:
            operator = i_2
        for i in range(1, self.n_qubits):
            if i == index:
                operator = np.kron(operator, operation)
            else:
                operator = np.kron(operator, i_2)
        return operator

    def to_numpy(self):
        return self.state

### Testing

These tests demonstrate that the quantum state is working correctly. Qubits can be measured, added, and transformed using built-in operations.

In [535]:
state = QuantumState([0.5, 0, 0.5, 0, 0.5, 0, 0.5, 0])
print(state)
print(state.measure(0))
print(state)

print("-" * 32)

state = QuantumState([1, 0, 0, 0])
state.H(1)
print(state)

state.CNOT(1, 0)
print(state)

q = QuantumState([1, 0])
state.add_qubit(q)
print(state)

print(state.measure(1))
print(state)

print("-" * 32)

state = QuantumState([1, 0, 0, 0])
state.H(0)
print(state)

state.CNOT(0, 1)
print(state)

print(state.measure(0))
print(state)

A = np.array([1, 0], dtype=complex)
B = np.array([1, 0], dtype=complex)

state = QuantumState.from_qubits(A, B)
print(state)


[0.5+0.j 0. +0.j 0.5+0.j 0. +0.j 0.5+0.j 0. +0.j 0.5+0.j 0. +0.j]
0
[0.5+0.j 0. +0.j 0.5+0.j 0. +0.j 0.5+0.j 0. +0.j 0.5+0.j 0. +0.j]
--------------------------------
[0.7071+0.j 0.    +0.j 0.7071+0.j 0.    +0.j]
[0.7071+0.j 0.    +0.j 0.    +0.j 0.7071+0.j]
[0.7071+0.j 0.    +0.j 0.    +0.j 0.    +0.j 0.    +0.j 0.    +0.j
 0.7071+0.j 0.    +0.j]
0
[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
--------------------------------
[0.7071+0.j 0.7071+0.j 0.    +0.j 0.    +0.j]
[0.7071+0.j 0.    +0.j 0.    +0.j 0.7071+0.j]
0
[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
